In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt

import file_ext_search as fes

import asd
import trace_proc

idx_file2 = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\PS3SLF_2021-09-17T110351Z_00047728.asd.acf.idx'
acf_file2 = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\PS3SLF_2021-09-17T110351Z_00047728.asd.acf'


idx_file = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\Prof2_abp56_Gd\PS3SLF_2024-07-07T174231Z_07793648.asd.acf.idx'
acf_file = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\SEB\Prof2_abp56_Gd\PS3SLF_2024-07-07T174231Z_07793648.asd.acf'

idx_files_2 = [idx_file2, idx_file]

datapath = 'D:\ABP49_Processing'
idx_files = fes.file_ext_search('.idx', datapath)

Searching *.idx files in directory:D:\ABP49_Processing


In [ ]:
save_to = r'C:\Data\test\ABP49_SLF.csv'

for idx_file in idx_files:
    pos_strings = trace_proc.get_pos_acf(idx_file)
    
    
    if os.path.exists(save_to):
        with open(save_to, 'a+') as f9:
            for line in pos_strings:
                f9.write(f'{line}\n')
    else:
        with open(save_to, 'w') as f9:
            f9.write('num,acf,asd,ISOdatetime,POSIXsec,lat,lon\n')
            
            for line in pos_strings:
                f9.write(f'{line}\n')

In [ ]:
print(pos_strings)

In [146]:
espg_code = 32634  # UTM34N

for file in idx_files_2[0:1]: 
    acf_file = file[:-4]   
    asd_obj_list = asd.ASDfile.create_from_idx_file(file)

    with open(acf_file, 'rb') as f1:
        buffer = f1.read()    
        
    traces = []
    start_times = []

    delay = 0  # ms
    trace_len = 250  # ms

    num_of_traces = 0
    dt = 0.082
    
    heave_array = None
    
    trace_num = 1
    for obj in asd_obj_list[:]:
    # for obj in asd_obj_list[:]:
        if num_of_traces > 1800:
            break
        else:
            asd.parse_xml_header(obj, buffer)
            asd.parse_bin_header(obj, buffer)

            for sounding in obj.soundings[:]:
                trace: trace_proc.Trace
                trace, envelope = trace_proc.proc_trace(trace_num, espg_code, sounding, obj, tracelen=trace_len, delay=delay)
                # traces.append(envelope)
                
                traces.append(trace)
                trace_num += 1


## SEGY

In [97]:
am = 0.7
bm = 0.4

print(int(np.rint(am)))
print(np.rint(bm))

1
0.0


In [88]:
import numpy as np

import matplotlib.pyplot as plt
import segyio


sample_no = len(traces[0])

class MySpec(object):
    def __init__(self):
        self.iline = 189  # default value for segyio
        self.xline = 193  # default value for segyio
        self.tracecount = 0
        self.samples = []  # sample times, a list
        self.ext_headers = 0
        self.format = 5  # 4-byte IEEE float
        self.endian = 'big'
        
path = r'C:\Data\test\mytest14_new_HeaveAttitude.sgy'

myspec = MySpec()

myspec.tracecount = len(traces)
myspec.samples = np.arange(delay,delay+trace_len+dt,dt)

with segyio.create(path, myspec) as f5:
    for num, trace in enumerate(traces):
        f5.trace[num] = trace

    # print(f5.header[0])
    
    

c:\Users\user\.conda\envs\id1054\Lib\site-packages\segyio\utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


In [149]:
# sample_no = len(traces[0])
from trace_proc import int_header

class MySpec(object):
    def __init__(self):
        self.iline = 189  # default value for segyio
        self.xline = 193  # default value for segyio
        self.tracecount = 0
        self.samples = []  # sample times, a list
        self.ext_headers = 0
        self.format = 5  # 4-byte IEEE float
        self.endian = 'big'
        
path = r'C:\Data\test\mytest16_new_HeaveAttitude.sgy'

myspec = MySpec()


traces_2 = traces[0:1000]

myspec.tracecount = len(traces_2)
myspec.samples = np.arange(delay,delay+trace_len+dt,dt)

with segyio.create(path, myspec) as f5:
    trace: trace_proc.Trace
    for num, trace in enumerate(traces_2):
        f5.trace[num] = trace.data
        
        for key in int_header.keys():
            f5.header[num][int_header[str(key)]] = trace.header[str(key)]
        

c:\Users\user\.conda\envs\id1054\Lib\site-packages\segyio\utils.py:23: RuntimeWarning: Implicit conversion from float64 to float32 (narrowing)
  warnings.warn(msg.format(x.dtype, dtype), RuntimeWarning)


In [131]:
ke = {1: 123}
# ke['am'] = 5

In [133]:
print(ke[1])

123


In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import segyio


class MySpec(object):
    def __init__(self):
        self.iline = 189  # default value for segyio
        self.xline = 193  # default value for segyio
        self.tracecount = 0
        self.samples = []  # sample times, a list
        self.ext_headers = 0
        self.format = 5  # 4-byte IEEE float
        self.endian = 'big'

def plot_rawtraces(raw_traces):
    clip = 1e+3
    vmin, vmax = -clip, clip

    # Figure
    figsize=(20, 20)
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=figsize, facecolor='w', edgecolor='k',
                        squeeze=False,
                        sharex=True)
    axs = axs.ravel()
    im = axs[0].imshow(raw_traces.T, cmap=plt.cm.seismic, vmin=vmin, vmax=vmax)
    plt.show()


path = r'D:\aa_yandexcloud\MyProjects\InspectingP70Data\P70_data\Profile-0_W1_SLF2109171103_LL_car.sgy'


with segyio.open(filename=path, mode="r",
                            strict = False,
                            ignore_geometry = True,
                            endian = 'big') as f:
    # print(np.shape(f.trace.raw[:]))
    # plot_rawtraces(f.trace.raw[:])
    
    print('\n_____SEG-Y FILE____\n')
    
    print(f'Sample format: {f.format}')
    print(f'Sample count: {len(f.samples)}')
    print(f'Trace count: {f.tracecount}')
    
    
    # modes are: .trace, .header, .iline/xline, .fast/slow, .depth_slice
    # .gather, .text, .bin
    
    # 2D seismic profiles are unstructured files
    # .trace and .header are the only modes available for unstructured files
    # traces are enumerated: 0..len(f.trace)
    # .trace mode - offers raw adressing of traces as they are laid out in the file
    # Reading a trace yields a numpy ndarray, and
    # reading multiple traces yields a generator of ndarray's
    
    # .header mode - accessing items yield header objects instead of numpy ndarray
    # Headers are dict-like objects, where
    # keys are integers, seismic unix-style keys and segyio enums
    
    print(f.samples)
    print(f.trace)
    print(f.trace[0])
    # print(f'{f.bin}')
    # print(f.text[0])
    # print(f'DELAY (109 byte): {f.header[0]['109']}')
    # print(len(f.header))
    # print(len(f.trace))
    # print('\n')
    # print(f.trace)
    # print(f.header)
    # print(len(f.samples))
    # print(type(np.fromiter(f.trace[:])))
    # print(np.fromiter(f.trace[:], dtype=np.dtype((float, len(f.samples)))))
    # plot_rawtraces(np.fromiter(f.trace[:], dtype=np.dtype((float, len(f.samples)))))